# LCCS Level 4 Change Test Notebook

A notebook to take the level 4 classification data for Australia in GeoTiff format and generate change results.

In [9]:
import argparse
import logging
import os
import sys
import yaml

import numpy
import xarray
from matplotlib import pyplot

# Import le_lccs modules
from le_lccs import le_ingest
from le_lccs import le_export

from le_lccs.le_classification import lccs_l3
from le_lccs.le_classification import lccs_l4
from le_lccs.le_classification import simple_change

ModuleNotFoundError: No module named 'ingest_base'

In [10]:
yaml_sites_file = os.path.abspath("../cultivated_ml/au_test_sites.yaml")

with open(yaml_sites_file, "r") as f:
    site_config = yaml.safe_load(f)

site_name = "Kangaroo Island"

extent = [site_config[site_name]["min_x"],
          site_config[site_name]["min_y"],
          site_config[site_name]["max_x"],
          site_config[site_name]["max_y"]]

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/dev/dea-notebooks/cultivated_ml/au_test_sites.yaml'

In [ ]:
# Set up dictionary to define size parameters
target_size = {}

target_size["target_min_x"] = extent[0]
target_size["target_max_x"] = extent[2]
target_size["target_min_y"] = extent[1]
target_size["target_max_y"] = extent[3]

target_size["target_pixel_size_x"] = 25
target_size["target_pixel_size_y"] = -25

target_size["target_crs"] = "EPSG:3577"

## Data Input

In [ ]:
# Set up dictionary with importers for each input layer

level4_tiff_2010 = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/lccs_2010_L4_beta-1.0.2.tif"
level4_tiff_2015 = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/lccs_2015_L4_beta-1.0.2.tif"

level4_bands = ["level3", "lifeform_veg_cat_l4a", "canopyco_veg_cat_l4d", "watersea_veg_cat_l4a",
                "waterstt_wat_cat_l4a", "waterper_wat_cat_l4d", "inttidal_wat_cat_l4a"]

# Add in all the level4 layers to ingest
input_layers_2010 = {}
input_layers_2015 = {}

for band, layer_name in enumerate(level4_bands):
    input_layers_2010[layer_name] = {}
    input_layers_2015[layer_name] = {}
    
    input_layers_2010[layer_name]["ingest_class"] = "gridded_ingest.LEIngestGDAL"
    input_layers_2010[layer_name]["input_file"] = level4_tiff_2010
    input_layers_2010[layer_name]["resampling"] = "mode"
    input_layers_2010[layer_name]["band"] = band + 1

    input_layers_2015[layer_name]["ingest_class"] = "gridded_ingest.LEIngestGDAL"
    input_layers_2015[layer_name]["input_file"] = level4_tiff_2015
    input_layers_2015[layer_name]["resampling"] = "mode"
    input_layers_2015[layer_name]["band"] = band + 1

## Additional layers
Add in additional layers below as needed. For example:
    
    input_layers_2010["cci_biomass"] = {}
    input_layers_2010["cci_biomass"]["ingest_class"] = "gridded_ingest.LEIngestGDAL"
    input_layers_2010["cci_biomass"]["input_file"] = "/path/to/file.tif"
    input_layers_2010["cci_biomass"]["resampling"] = "nearest"
    

In [ ]:
input_layers_2010["cci_biomass"] = {}
input_layers_2010["cci_biomass"]["ingest_class"] = "gridded_ingest.LEIngestGDAL"
input_layers_2010["cci_biomass"]["input_file"] = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/S3436_136_abg2010.tif"
input_layers_2010["cci_biomass"]["resampling"] = "nearest"

input_layers_2015["cci_biomass"] = {}
input_layers_2015["cci_biomass"]["ingest_class"] = "gridded_ingest.LEIngestGDAL"
input_layers_2015["cci_biomass"]["input_file"] = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/S30E130_agb2017.tif"
input_layers_2015["cci_biomass"]["resampling"] = "nearest"

In [ ]:
# Read each into xarray and save to list
logging.info("Reading in data for 2010")
variables_xarray_list_2010 = []

print("Loading 2010 Data")

for var_name, ingest_parameters in input_layers_2010.items():
    # TODO: Need to write a wrapper to make recursive calls of `getattr` nicer.
    # Don't run any of the 'xarray_maths' classes yet - nead to load all other
    # variables first
    print("Loading: {}".format(var_name))
    import_class_name = ingest_parameters["ingest_class"]
        # Get the class to use for importing
    if import_class_name != "xarray_maths":
        import_class = getattr(getattr(le_ingest, import_class_name.split(".")[0]),
                              import_class_name.split(".")[1])
        # Set up instance of import class
        import_obj = import_class(**target_size)

        # Import
        ingest_parameters["variable_name"] = var_name
        variables_xarray = import_obj.read_to_xarray(**ingest_parameters)
        variables_xarray_list_2010.append(variables_xarray)

# Merge to a single dataset
level4_2010 = xarray.merge(variables_xarray_list_2010)

# Read each into xarray and save to list
logging.info("Reading in data for 2015")
variables_xarray_list_2015 = []

print("Loading 2015 Data")

for var_name, ingest_parameters in input_layers_2015.items():
    # TODO: Need to write a wrapper to make recursive calls of `getattr` nicer.
    # Don't run any of the 'xarray_maths' classes yet - nead to load all other
    # variables first
    print("Loading: {}".format(var_name))
    import_class_name = ingest_parameters["ingest_class"]
        # Get the class to use for importing
    if import_class_name != "xarray_maths":
        import_class = getattr(getattr(le_ingest, import_class_name.split(".")[0]),
                              import_class_name.split(".")[1])
        # Set up instance of import class
        import_obj = import_class(**target_size)

        # Import
        ingest_parameters["variable_name"] = var_name
        variables_xarray = import_obj.read_to_xarray(**ingest_parameters)
        variables_xarray_list_2015.append(variables_xarray)

# Merge to a single dataset
level4_2015 = xarray.merge(variables_xarray_list_2015)

## Change Detection

### Simple Level 4 Change

In [ ]:
l4_change_codes = simple_change.get_lccs_l4_change_codes(level4_2010, level4_2015, ignore_no_change=False)
l4_change_codes

### Biomass Change

In [ ]:
biomass_diff = level4_2010["cci_biomass"].values.astype(numpy.float32) - level4_2015["cci_biomass"].values.astype(numpy.float32)

# Convert numpy array with change values into an xarray Dataset
biomass_diff_xarray = xarray.Dataset(
    {"cci_biomass_diff" : (level4_2010["cci_biomass"].dims, biomass_diff)},
    coords=level4_2010.coords)

In [ ]:
# Merge into a single xarray
l4_change_codes = xarray.merge([l4_change_codes, biomass_diff_xarray])

In [ ]:
# Export data as a GeoTiff

export_parameters = {}
export_parameters["output_file"] = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/{}_2010-2015_l4_change.tif".format(site_name.replace(" ","_"))
# Use a float for output and cast all output layer to this
export_parameters["out_dtype"] = numpy.float32
export_parameters["cast_to_out_dtype"] = True
# Get export class to use
export_class = le_export.gridded_export.LEExportGDAL

export_obj = export_class(**target_size)

# Write out xarray
export_obj.write_xarray(l4_change_codes, **export_parameters)

### Evidence Based Change - Wildfire

Wildfire has change code 1.

* Need Level3 to have stayed as 112
* Canopy Cover catergory 10 -> 16

In [ ]:
wild_fire = numpy.where((l4_change_codes["level3_change"].values == 112112) &
                        (l4_change_codes["canopyco_veg_cat_l4d_change"].values == 1016), 1, 0)

In [ ]:
# Convert to xarray
wild_fire_xarray = xarray.Dataset(
    {"wild_fire" : (level4_2010["level3"].dims, wild_fire)},
    coords=level4_2010.coords)

In [ ]:
pyplot.imshow(wild_fire)

In [ ]:
# Export data as a GeoTiff
export_parameters = {}
export_parameters["output_file"] = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/{}_2010-2015_wild_fire.tif".format(site_name.replace(" ","_"))
# Use a float for output and cast all output layer to this
export_parameters["out_dtype"] = numpy.uint16
export_parameters["cast_to_out_dtype"] = True
# Get export class to use
export_class = le_export.gridded_export.LEExportGDAL

export_obj = export_class(**target_size)

# Write out xarray
export_obj.write_xarray(wild_fire_xarray, **export_parameters)

### Evidence Based Change - Insect disease

Insect disease has change code 2

In [ ]:
insect_disease = numpy.where((l4_change_codes["level3_change"].values == 112112) &
                        (l4_change_codes["canopyco_veg_cat_l4d_change"].values == 1016), 1, 0)

# Convert to xarray
insect_disease_xarray = xarray.Dataset(
    {"insect_disease" : (level4_2010["level3"].dims, insect_disease)},
    coords=level4_2010.coords)

pyplot.imshow(insect_disease)

# Export data as a GeoTiff
export_parameters = {}
export_parameters["output_file"] = "/Users/richard/Dropbox (EOED)/Data_Projects/DEA_LandCoverMonitoring/Change/{}_2010-2015_insect_disease.tif".format(site_name.replace(" ","_"))
# Use a float for output and cast all output layer to this
export_parameters["out_dtype"] = numpy.uint16
export_parameters["cast_to_out_dtype"] = True
# Get export class to use
export_class = le_export.gridded_export.LEExportGDAL

export_obj = export_class(**target_size)

# Write out xarray
export_obj.write_xarray(insect_disease_xarray, **export_parameters)